# Testing my implementation of VQ-VAE 

In [ ]:
import sys
import importlib

# data manipulation
import pandas as pd

# pytorch specific
import torch

# project specific
sys.path.append('../')
from src import config
from src.utils import experiment, helpers


pd.options.display.width = 1000
DEVICE = torch.device(config["DEVICE"])

import random


## We first set the grid of parametter we want to explore

In [ ]:
# or we  use a pre loaded dataset to save time
data_params = '../workfiles/light_BRCA_ds.pkl'

model_params = {
    "dropout" : 0.2,
    "latent_dim" : 64,
    "variational" : "VQ-VAE",
    "convolution": False,
    "transformer" : False
    }



In [ ]:
# This one is absurdly clumsy but that's the only way i could do it.

variational_param_ = {
    "variational": [ # constraint : embedding_dim <= num_embeddings
        (("embedding_dim", 16), ("num_embeddings", 64)),
        (("embedding_dim", 16), ("num_embeddings", 128)),
        (("embedding_dim", 16), ("num_embeddings", 256)),
        (("embedding_dim", 16), ("num_embeddings", 512)),
        (("embedding_dim", 32), ("num_embeddings", 64)),
        (("embedding_dim", 32), ("num_embeddings", 128)),
        (("embedding_dim", 32), ("num_embeddings", 256)),
        (("embedding_dim", 32), ("num_embeddings", 512)),
        (("embedding_dim", 64), ("num_embeddings", 64)),
        (("embedding_dim", 64), ("num_embeddings", 128)),
        (("embedding_dim", 64), ("num_embeddings", 256)),
        (("embedding_dim", 64), ("num_embeddings", 512)),
        (("embedding_dim", 128), ("num_embeddings", 128)),
        (("embedding_dim", 128), ("num_embeddings", 256)),
        (("embedding_dim", 128), ("num_embeddings", 512))]}


In [ ]:
importlib.reload(helpers)


dynamic_params = {
    "latent_dim": [16, 32, 64, 128],
    "variational": [],
}

for commitment_cost in [0.2, 0.4, 0.6, 0.8, 1]:
    for (param1, param2) in variational_param_['variational']:
        dynamic_params["variational"].append((("variational", "VQ-VAE"),("commitment_cost", commitment_cost),param1, param2))


dynamic_params['variational'] = dynamic_params['variational'] + [None]*len(dynamic_params['variational']) + ["VAE"]*len(dynamic_params['variational'])

configurations = helpers.generate_config(model_params, dynamic_params)
print("number of config to be evaluated:",len(configurations))

In [ ]:
random.choice(configurations)

In [ ]:
importlib.reload(experiment)
experiments = []

for config in configurations:
    e = experiment.Experiment(data_param=data_params, model_param=config)
    e.run()

    experiment_data = {**config,'score': e.metric}
    experiments.append(experiment_data)